## Problem 1: Classification Problem

In [ ]:
import torch

#### Q1 (10 points): Implement Instance Normalization

#### Q2 (10 points): Implement Dropout

#### Q3 (10 points): Implement Softmax Cross Entropy

#### Q4 (10 points): Implement Tanh

#### Q5 (10 points): Build and Train a Classification Model

#### Q6 (25 points): InstanceNorm vs BatchNorm

#### Q7 (25 points): InstanceNorm With and Without Dropout